### Imports

In [ ]:
import mediapipe as mp
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
import mediapipe as mp
from itertools import combinations
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


### Function to calculate angles and head poses

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from itertools import combinations

# Initialize MediaPipe models
mp_pose = mp.solutions.pose.Pose(static_image_mode=True)
mp_fm = mp.solutions.face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.35)

def calculate_angle(p1, p2, p3):
    """Calculate the angle between three points."""
    a = np.array(p1)
    b = np.array(p2)
    c = np.array(p3)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

def process_image(image_path):
    """Process a single image for pose angles and head angles."""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image with pose
    pose_results = mp_pose.process(image)
    pose_angles = []
    if pose_results.pose_landmarks:
        # Ensure landmarks are correctly referenced
        landmarks = np.array([(lm.x, lm.y, lm.z) for lm in pose_results.pose_landmarks.landmark[:25]])
        for combo in combinations(range(len(landmarks)), 3):
            angle = calculate_angle(landmarks[combo[0]], landmarks[combo[1]], landmarks[combo[2]])
            pose_angles.append(angle)
    

    # Process the image with face mesh
    face_results = mp_fm.process(image)
    head_angles = []
    if face_results.multi_face_landmarks:
        face_3d = []
        face_2d = []
        img_h, img_w, _ = image.shape
        for face_landmarks in face_results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in [33, 263, 61, 199]:
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])

            # Solve PnP
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)
            cam_matrix = np.array([[img_w, 0, img_h / 2], [0, img_w, img_w / 2], [0, 0, 1]])
            dist_matrix = np.zeros((4, 1), dtype=np.float64)
            _, rot_vec, _ = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
            rmat, _ = cv2.Rodrigues(rot_vec)
            angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
            head_angles = [angles[0] * 360, angles[1] * 360]

    return pose_angles, head_angles

def process_folder(folder_path, label):
    """Process all images in a folder and associate them with a label."""
    folder_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            pose_angles, head_angles = process_image(image_path)
            folder_data.append((pose_angles + head_angles, label))
    return folder_data


### Loading and processing data

In [ ]:
folder_1_path = '/home/vaishnavi/MLPR_Project/Integrated_data/1'
folder_3_path = '/home/vaishnavi/MLPR_Project/Integrated_data/3'
folder_5_path = '/home/vaishnavi/MLPR_Project/Integrated_data/5'

data_1 = process_folder(folder_1_path, 0)  # Label 0 for folder 1
data_3 = process_folder(folder_3_path, 1)  # Label 1 for folder 3
data_5 = process_folder(folder_5_path, 2)  # Label 2 for folder 5

all_data = data_1 + data_3 + data_5
np.random.shuffle(all_data)

In [ ]:
max_length = max(len(x) for x, _ in all_data)
padded_data = [(np.pad(x, (0, max_length - len(x)), 'constant'), y) for x, y in all_data]

X = np.array([x for x, _ in padded_data])
y = np.array([y for _, y in padded_data])


### Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### XG Boost Classifier

In [ ]:
# Create and train the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, num_estimators = 300, learning_rate=0.1, max_depth=5)
xgb_clf.fit(X_train, y_train)


### Evaluation

In [ ]:
y_pred_xgb = xgb_clf.predict(X_test)

In [ ]:
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Test Accuracy: {accuracy_xgb}')

In [ ]:
precision = precision_score(y_test, y_pred_xgb, average='weighted')
recall = recall_score(y_test, y_pred_xgb, average='weighted')
f1 = f1_score(y_test, y_pred_xgb, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred_xgb)


In [ ]:
print(f'XGBoost Test Accuracy: {accuracy_xgb}')
print(f'XGBoost Precision: {precision}')
print(f'XGBoost Recall: {recall}')
print(f'XGBoost F1 Score: {f1}')


In [ ]:
# Calculate the confusion matrix again if needed
conf_matrix = confusion_matrix(y_test, y_pred_xgb)

# Plotting using Seaborn
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=[1, 3, 5], yticklabels=[1, 3, 5])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
xgb_clf.save_model('xgboost_model.bin')

### SVM

In [ ]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

svm_accuracy = accuracy_score(y_test, y_pred_svm)
print(f"SVM Test Accuracy: {svm_accuracy}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Test Accuracy: {rf_accuracy}")


### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Test Accuracy: {rf_accuracy}")


### One hot encoding for neural network

In [ ]:
y_one_hot = tf.keras.utils.to_categorical(y, 3)

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

### Neural Network

In [ ]:
from tensorflow.keras.layers import BatchNormalization
BatchNormalization()

def create_model(input_size):
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_size,)),
        Dropout(0.3),
        BatchNormalization(),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        Dropout(0.3),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(3, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

input_size = X_train.shape[1]
model = create_model(input_size)
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)


### Evaluation of neural network

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")
